# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
from citipy import citipy

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Reference output_data_file to load Dataframe
csv_path = "../output_data/cities.csv"
#Reference output_data_file
city_weather_df = pd.read_csv(csv_path)
city_weather_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure Gmaps
gmaps.configure(api_key=g_key)

In [4]:
#Locations
locations = city_weather_df[["Lat", "Lng"]]
#Humidity
humidity = city_weather_df["Humidity"].astype(float)
#Figure Layout
fig = gmaps.figure()
#Heatmap Layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=100,
                                   point_radius=3)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition
# A max temperature lower than 85 degrees but higher than 75
# Wind speed less than 5 mph
# Zero cloudiness.

vacation_df = pd.DataFrame(city_weather_df, columns = ["City", "Country",  "Max Temp", "Wind Speed", "Cloudiness", "Lat", "Lng"])

max_temp = (vacation_df["Max Temp"] <= 85) & (vacation_df["Max Temp"] > 75)
wind_speed = vacation_df["Wind Speed"] < 5
cloudiness = vacation_df["Cloudiness"] <= 0

# New vacation_df that only contains [max_temp & wind_speed & cloudiness]
vacation_df[max_temp & wind_speed & cloudiness]

,City,Country,Max Temp,Wind Speed,Cloudiness,Lat,Lng
125,arraial do cabo,BR,84.2,4.70,0,-22.97,-42.02
139,vaini,IN,78.9,1.59,0,15.34,74.49


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Store into variable named hotel_df.
hotel_df = vacation_df[["City", "Country", "Lat", "Lng"]].copy()
#Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,ostrovnoy,RU,68.05,39.51,
1,mahebourg,MU,-20.41,57.70,
2,qaanaaq,GL,77.48,-69.36,
3,zhuhai,CN,40.71,112.04,
4,cape town,ZA,-33.93,18.42,


In [ ]:
#Params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

#Create Loop
for index, row in hotel_df.iterrows():
    #Get Lat & Lng from df
    Lat= row["Lat"]
    Long = row["Lng"]

    #Change location with each iteration while leaving original params in place
    params["location"] = f"{Lat},{Long}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #Make request abd convert to JSON
    response = requests.get(base_url, params = params).json()
    try:
        #Store First Hotel Result into the dataframe
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("No Results Found")
    
hotel_df.head()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig